<a href="https://colab.research.google.com/github/styndall/nlp_tools/blob/master/Basic_Character_Model_trainer_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


In [ ]:
# some quick stuff to clean up data
def get_dict_from_column(df, col_name):
  col = df[col_name]
  col_dict = dict(zip(df[col_name].unique(), range(1, df[col_name].nunique()+1)))
  for k in col_dict:
    if col_dict[k] == '':
      col_dict[k] = 0
  return col_dict

def get_all_col_dicts(df):
  col_dicts = {}
  for col in df:
    col_dicts[col] = get_dict_from_column(df, col)
  return col_dicts

def replace_cols_with_numerical(df):
  col_dicts = get_all_col_dicts(df)
  transformed_df = pd.DataFrame()
  for col in df.columns:
    transformed_df[col] = df[col].map(col_dicts[col])

  return transformed_df

def get_original_df_from_numerical(df, col_dict):
  reversed_col_dict = {}
  for col in col_dict:
    reversed_col_dict[col] = reverse(col_dict[col])

def get_all_reverse_dicts(col_dicts):
  reverse_dicts = {}
  for col in col_dicts.keys():
    reverse_dicts[col] = reverse_dict(col_dicts[col])
  
  return reverse_dicts

def reverse_dict(col_dict):
  reverse_dict = {v:k for k, v in col_dict.items()}
  reverse_dict[0] = ''
  return reverse_dict

In [ ]:
def get_diff_col(col):
  out = [col[0]]
  for i in range(1, len(col)):
    out.append(col[i] - col[i-1])

  return out

def reverse_get_diff_col(col, min_val):
  if min_val != 0:
    for i in range(len(col)):
      col[i] = col[i] + min_val
  out = [col[0]]
  for i in range(1, len(col)):
    out.append(col[i] + out[i-1])

  return out

def blur_time_col(time_col, seconds=2):
  # this is a list of epoch timestamps, alter at random by a few seconds
  final_col = [i + random.randint(0, seconds) for i in time_col]
  

  return final_col

    

In [28]:
# XXX get training data, import as string
with open("gdrive/My Drive/british_pub_names/pub_name_list.txt") as f:
  data = f.read()
data[0:25]


'Anchor Bankside\nThe Bedfo'

In [29]:
print(data[0:100])
len(data[15:20])

Anchor Bankside
The Bedford
Bell Savage Inn
The Bishop's Finger
The Black Friar, Blackfriars
Black L


5

In [30]:
data_as_list = data
unique_elements = list(set(data_as_list))
ele2idx = {}
idx2ele = {}
for i in range(len(unique_elements)):
  ele2idx[unique_elements[i]] = i
  idx2ele[i] = unique_elements[i]




In [31]:
transformed_data_as_list = [ele2idx[x] for x in data_as_list]
transformed_data_as_list[0:25]
print(len(transformed_data_as_list))

7593


In [7]:
# set up everything for training (I should really make a simple library
# and just import it)
def train(data_col, training_epochs = 10, model_name=None):
  distinct_values = len(set(data_col)) + 1

  # sequence_length is how far we look back in our list to predict the
  # nth character, i.e. n-10..n-1 is the sequence, n is the predicted char
  sequence_length = 7

  examples_per_epoch = len(data_col)//(sequence_length+1)
  col_dataset = tf.data.Dataset.from_tensor_slices(data_col)
  #split into paired sequences, target with the next sequence item
  sequences = col_dataset.batch(sequence_length+1, drop_remainder=True)
  dataset = sequences.map(split_input_target_seq)

  # set up batches
  batch_size = 64
  buffer_size = 10000

  #shuffle, do we need to do this for non-char-type sequences?
  dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)


  #set up some model values

  embedding_dims = 256 #pretty standard dim count
  rnn_units = 1024


  model = build_model(distinct_values, embedding_dims, rnn_units, batch_size)
  model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])


  # set up checkpoints with sensible names
  if model_name:
    checkpoint_dir = "gdrive/My Drive/tf_models/{}/checkpoints".format(model_name)
  else:
    checkpoint_dir = "gdrive/My Drive/tf_models/checkpoints"
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

  checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='accuracy',
    save_best_only=True,
    save_weights_only=True)
  
  # TRAIN!!!
  epochs = training_epochs
  history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])

  predictor_model = build_model(distinct_values, embedding_dims, rnn_units, batch_size=1)
  predictor_model.set_weights(model.get_weights())

  return predictor_model





# set the lstm model up
def build_model(distinct_values, embedding_dims, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(distinct_values, embedding_dims,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True),
    tf.keras.layers.Dense(distinct_values)
  ])
  return model

def split_input_target_seq(seq):
  input = seq[:-1]
  target = seq[1:]
  return input, target

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


def get_latest_model(vocab_size, embedding_dims, rnn_units):
  model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
  model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
  model.build(tf.TensorShape([1, None]))
  return model

def generate_seq(model, start_seq, target_length=100, temperature=1.0):

  # Number of elements to generate
  num_generate = target_length
  
  input_eval = tf.expand_dims(start_seq, 0)

  seq_generated = []

  #how crazy do we want the results? low = less crazy, high = more


  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    seq_generated.append(predicted_id)

  return seq_generated

In [ ]:
# Idea for per-column training: vocab is strings, basically - combinations of the rows, turned to strings
# for the cols with large nuniques, use a histogram or other dimensionality reducing something

In [22]:
model = train(transformed_data_as_list, training_epochs=100)

Epoch 1/100
15/15 [==============================] - 0s 28ms/step - loss: 3.8419 - accuracy: 0.0930
Epoch 2/100
15/15 [==============================] - 0s 30ms/step - loss: 3.3638 - accuracy: 0.1301
Epoch 3/100
15/15 [==============================] - 0s 29ms/step - loss: 3.1781 - accuracy: 0.1964
Epoch 4/100
15/15 [==============================] - 0s 31ms/step - loss: 2.8213 - accuracy: 0.2586
Epoch 5/100
15/15 [==============================] - 0s 29ms/step - loss: 2.5213 - accuracy: 0.2909
Epoch 6/100
15/15 [==============================] - 0s 29ms/step - loss: 2.3804 - accuracy: 0.3137
Epoch 7/100
15/15 [==============================] - 0s 30ms/step - loss: 2.2958 - accuracy: 0.3290
Epoch 8/100
15/15 [==============================] - 0s 30ms/step - loss: 2.2198 - accuracy: 0.3475
Epoch 9/100
15/15 [==============================] - 1s 52ms/step - loss: 2.1586 - accuracy: 0.3688
Epoch 10/100
15/15 [==============================] - 0s 23ms/step - loss: 2.1035 - accuracy: 0.3826

In [24]:
model.save("gdrive/My Drive/british_pub_names/model/1")

INFO:tensorflow:Assets written to: gdrive/My Drive/british_pub_names/model/1/assets


In [8]:
def predict_from_data(data_column, training_epochs=100, number_of_predictions=6000):
  #train the model
  print("Starting training")
  model = train(data_column, training_epochs=training_epochs)
  print("Finished training, predicting")
  predictions = generate_seq(model, data_column[0:60], target_length=number_of_predictions)
  print("Finished predicting")
  return predictions

def data_from_transformed(data_list, transform_dict):
  final_data = []
  for i in data_list:
    final_data.append(transform_dict.get(i, 0))
  return final_data

In [9]:
def train_and_gen_csv(input_data, reverse_dict, training_epochs=10):
  generated_frame = pd.DataFrame()

  print("Training and predicting")
  prediction = predict_from_data(input_data, training_epochs=training_epochs)

  print("Reconstructing data")

  final = [reverse_dict[i] for i in prediction]


  return final


In [ ]:
# preprocessing - for cols with .nunique > 10, instead, transform to difference from prior value

In [32]:
# generate new data_list
generated_data_as_list = train_and_gen_csv(transformed_data_as_list, idx2ele, training_epochs=150)



Training and predicting
Starting training
Epoch 1/150
14/14 [==============================] - 0s 28ms/step - loss: 3.9897 - accuracy: 0.0770
Epoch 2/150
14/14 [==============================] - 0s 31ms/step - loss: 3.4144 - accuracy: 0.1137
Epoch 3/150
14/14 [==============================] - 0s 28ms/step - loss: 3.2636 - accuracy: 0.1513
Epoch 4/150
14/14 [==============================] - 0s 29ms/step - loss: 3.0175 - accuracy: 0.2226
Epoch 5/150
14/14 [==============================] - 0s 30ms/step - loss: 2.6777 - accuracy: 0.2797
Epoch 6/150
14/14 [==============================] - 0s 31ms/step - loss: 2.4592 - accuracy: 0.3037
Epoch 7/150
14/14 [==============================] - 0s 30ms/step - loss: 2.3411 - accuracy: 0.3170
Epoch 8/150
14/14 [==============================] - 1s 55ms/step - loss: 2.2721 - accuracy: 0.3371
Epoch 9/150
14/14 [==============================] - 0s 25ms/step - loss: 2.1956 - accuracy: 0.3611
Epoch 10/150
14/14 [==============================] - 0s 3

In [33]:
print("".join(generated_data_as_list))


ger
The Toote
Casedon
The Dovent Garden
Devons
Leatshire
Lord Shears
Black Horses, Ismith
The and Navern
Fitzrovia				
Scots House, Southwark, Fitzrovia				
Crown, Coventer				
The flask				
The Flen
Ye Old Belgs Arms, Novendon Apppttyle, Ke Arms
Quree and Dred, Maida Vale
Whe Crown, Islington
The Dray Tavern
George and Dragon, Fitzrovia
Swan & Kenning Scotuthwark
White Swan, Hight Bell
Leatherd Navern, Fitzrovia			
The Boot, Chelsea
The Goate
Britre Hog
The Bishop's Fishall
The Hleworth
The Colornsey
Queen's Her
Colehite Lion, House, Southwark
Fox Inn, Richmond
Denmarken ury
The Roeh, Brixter
The Olde Mitzrovia
Sgar Botthese
Two Covern Castle, Hammersmith
Salutty Sarden
The Duke olden
Barlton Cour
Ters, Covent Garden
Ye Old Bells, Fulham
The Wat
Dirth
Mawson Arms
Arms
Spandra, Dick
Eavern
The Old Bulllet
King Edwingtorse
The Harp
Bricklarnes
Wheewn and Anchor Bernchor, Harnes
Whe Wheatsheaf, Fitzrovia			
The Harp
Lather
Rayne
The Niardy Hanwell Arms
Camlico
The Bishop's Fish Thabert
A

In [ ]:
generated_data = pd.DataFrame(rows, columns=["event_type", "user_id", "user_name", "company_id", "company_name", "call_id"])
time_col = blur_time_col(time_col[0:len(generated_data)])
generated_data['timestamp'] = time_col

In [ ]:
generated_data[0:20]

,event_type,user_id,user_name,company_id,company_name,call_id,timestamp
0,CallLeftEvent,e3a89054-732a-4c86-98d3-bcfe7918b092,John Gamble,570afd99-616b-4649-b899-50dcb1d8af77,"Smith, Patton and Robinson",0001987e-bf61-4d40-a979-ca97c3c6fe0b,1459488133
1,CallLeftEvent,4ceed7a9-30a1-40c5-9cc8-afcd5db5fae6,Natalie Barron,570afd99-616b-4649-b899-50dcb1d8af77,"Smith, Patton and Robinson",0001987e-bf61-4d40-a979-ca97c3c6fe0b,1459488133
2,CallCreatedEvent,c9c1c9e8-3788-4e60-a87e-9f8752d9b730,Jack Flores,7c015f5c-93cd-4f06-b9c6-c6dfdbfd3c1e,Leblanc PLC,b31d9a92-0867-4b78-ad5b-aed2e9fe2525,1459512944
3,CallLeftEvent,c9c1c9e8-3788-4e60-a87e-9f8752d9b730,Jack Flores,7c015f5c-93cd-4f06-b9c6-c6dfdbfd3c1e,Leblanc PLC,b31d9a92-0867-4b78-ad5b-aed2e9fe2525,1459512942
4,CallCreatedEvent,9acaa899-e559-4600-b376-af6b211833bf,James Reynolds,570afd99-616b-4649-b899-50dcb1d8af77,"Smith, Patton and Robinson",b321f1c8-43b6-4284-9f50-2c5e4c70a0d5,1459513453
5,CallLeftEvent,9acaa899-e559-4600-b376-af6b211833bf,James Reynolds,570afd99-616b-4649-b899-50dcb1d8af77,"Smith, Patton and Robinson",b321f1c8-43b6-4284-9f50-2c5e4c70a0d5,1459513453
6,CallCreatedEvent,6eed472b-87a7-4203-9bf2-fb0a75383a64,Latoya Schneider,a7306837-37e0-4f62-b1a4-5e257a0203e5,"Brown, Duncan and Jones",b324b5d1-62d2-415f-83cf-cabf05e9f495,1459514077
7,CallLeftEvent,6eed472b-87a7-4203-9bf2-fb0a75383a64,Latoya Schneider,a7306837-37e0-4f62-b1a4-5e257a0203e5,"Brown, Duncan and Jones",b324b5d1-62d2-415f-83cf-cabf05e9f495,1459514074
8,CallCreatedEvent,2e21e535-8ecb-45b3-a92a-a3476e08b5d6,James Mckenzie,d467376c-70b5-4a16-8936-6cd73d50d70c,Gilbert Group,b328c9a0-d813-4f09-9a2c-3dc3a922c873,1459514395
9,CallJoinedEvent,50db4ef5-e202-409b-a28a-6876d9469b4b,Hannah Freeman,d467376c-70b5-4a16-8936-6cd73d50d70c,Gilbert Group,b328c9a0-d813-4f09-9a2c-3dc3a922c873,1459514396


In [ ]:
generated_data.to_csv("gdrive/My Drive/tonic/call_model_ordered_by_call_id.csv")

In [ ]:
generated_data.to_csv("gdrive/My Drive/tonic/call_model_trained_.csv")